In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sps
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
%matplotlib inline
start_time = pd.datetime.now()

In [ ]:
%%time
alldata = pd.read_csv('transactions_3.20.csv')

In [ ]:
list(alldata.columns)

In [ ]:
alldata[['mean','Amount','Fraud','max','sum','maxstate_day90']]

In [ ]:
A=['count','sum']
B=['card_','merch_']
C=['day1','day3']
D=[]
for a in A:
    for b in B:
        for c in C:
            D.append(a+b+c)
D.append('count_merch_day0')
D.append('count')
D.append('sum_merch_day0')
D.append('sum')

In [ ]:
D

In [ ]:
E=['day7','day14','day30','day90']
F=[]
for a in A:
    for b in B:
        for c in E:
            F.append(a+b+c)
F

# velocity change variables

In [ ]:
for i in D:
    for j in F:
        if j=='countcard_day7' or j=='countmerch_day7' or j=='summerch_day7':
            alldata[i+'_over_'+j]=alldata[i]/(alldata[j]/7)
        elif j=='countcard_day14' or j=='countmerch_day14' or j=='summerch_day14':
            alldata[i+'_over_'+j]=alldata[i]/(alldata[j]/14)
        elif j=='countcard_day30' or j=='countmerch_day30' or j=='summerch_day30':
            alldata[i+'_over_'+j]=alldata[i]/(alldata[j]/30)
        else:
            alldata[i+'_over_'+j]=alldata[i]/(alldata[j]/90)

In [ ]:
alldata

In [ ]:

list(alldata.columns)

In [ ]:
alldata.shape

In [ ]:
#list=['Date','Date_previous_mc','Date_previous_card','Date_previous_zip','Date_previous_merch','Date_previous_state']
#for i in list:
alldata['Date']=pd.to_datetime(alldata['Date'])

In [ ]:
mydata=alldata[alldata['Date']<'2010-11-01']

In [ ]:
mydata.drop(['Recnum'],axis=1,inplace=True)
mydata['RANDOM']=np.random.ranf(len(mydata))
mydata

In [ ]:
backup=mydata
#backup
#mydata=backup
#mydata

In [ ]:
mydata.describe(include='all')

In [ ]:

list(mydata.count())

In [ ]:
mydata.max()

In [ ]:
variables=mydata.drop(['Fraud','Cardnum','Date','Merchnum','Merch description','Merch state',\
                            'Merch zip','Transtype','Amount','order_merch','order_card','order_mc',\
                               'order_zip','order_state',],axis=1).columns

In [ ]:
mydata=mydata.drop(['Cardnum','Date','Merchnum','Merch description','Merch state',\
                           'Merch zip','Transtype','Amount','order_merch','order_card',\
                             'order_mc','order_zip','order_state',],axis=1)

In [ ]:
nvars=len(variables)
print(nvars)

In [ ]:
list(mydata.columns)

In [ ]:
goods=mydata[mydata['Fraud']==0]
goods.shape

In [ ]:
bads=mydata[mydata['Fraud']==1]
bads.shape

In [ ]:
numgoods=len(goods)
numbads=len(bads)

In [ ]:

KS=pd.DataFrame(np.zeros((nvars+2,3)))
i=0
for column in mydata:
    print(i,column)
    KS.iloc[i,0]=column
    i=i+1
KS

In [ ]:
KS.columns = ['field','ks','FDR']
KS

In [ ]:
KS['field'][118:199]

In [ ]:
%%time
n=0
for i in mydata.columns:
    x = goods[i]
    y = bads[i] 
    KS['ks'][n] = sps.ks_2samp(x, y)[0]
    n+=1

In [ ]:
KS

In [ ]:
KS.sort_values(by=['ks'],ascending=False,inplace=True)
KS

In [ ]:
topRows= int(round(len(mydata)*0.03))
j=0
for column in mydata:
    temp1=mydata.sort_values(column,ascending=False).head(topRows)
    temp2=mydata.sort_values(column,ascending=True).head(topRows)
    needed1=temp1.loc[:,'Fraud']
    needed2=temp2.loc[:,'Fraud']
    FDR1=sum(needed1)/numbads
    FDR2=sum(needed2)/numbads
    FDRate=np.maximum(FDR1,FDR2)
    #print(j,column,FDR1,FDR2,FDRate)
    KS.loc[j,'FDR']=FDRate
    j=j+1

In [ ]:
KS['rank_ks']=KS['ks'].rank(ascending=True)
KS['rank_FDR']=KS['FDR'].rank(ascending=True)
KS.sort_values(by=['ks'],ascending=False,inplace=True)
KS.to_csv('KS.csv')
KS.sort_values(by=['FDR'],ascending=False,inplace=True)
KS.to_csv('FDR.csv')
KS.head(20)

In [ ]:
KS['average_rank']=(KS['rank_ks']+KS['rank_FDR'])/2
KS.sort_values(by=['average_rank'],ascending=False, inplace=True)
KS.reset_index(drop=True)

In [ ]:
num_keep=round(nvars/2)

In [ ]:
cols_keep=KS['field'][1:num_keep]
cols_keep=list(cols_keep)

In [ ]:
cols_keep.insert(0,'Fraud')
cols_keep.insert(0,'Recnums')
vars_keep = alldata.filter(cols_keep,axis=1)
vars_keep.shape


In [ ]:
stop_time=pd.datetime.now()
print(stop_time-start_time)

In [ ]:
vars_keep.to_csv('C:/Users/clair/Desktop/USC/DSO562 Fraud/project2/data100.csv')